In [ ]:
%%capture
!pip install langgraph groq python-dotenv PyPDF2 langchain-groq

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
resume_path = "/content/drive/MyDrive/DG'S Resume.pdf"
jd_path = "/content/drive/MyDrive/jd1.txt"

## Extracting Resume and JD

In [ ]:
from PyPDF2 import PdfReader

def extract_text_from_pdf_pypdf2(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            extracted_page_text = page.extract_text()
            if extracted_page_text:
                text += extracted_page_text + "\n"
        return text
    except FileNotFoundError:
        return f"Error: The file at path '{pdf_path}' was not found."
    except Exception as e:
        return f"An error occurred: {e}"

resume = extract_text_from_pdf_pypdf2(resume_path)

In [ ]:
def extract_text_from_txt(txt_path):
    try:
        with open(txt_path, "r", encoding="utf-8") as f:
            text = f.read()
        return text
    except FileNotFoundError:
        return f"Error: The file at path '{txt_path}' was not found."
    except Exception as e:
        return f"An error occurred: {e}"

jd = extract_text_from_txt(jd_path)

In [ ]:
import os

GROQ_API_KEY="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"
os.environ['GROQ_API_KEY'] = GROQ_API_KEY



## First Iteration

In [ ]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from groq import Groq

# Load API Key
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

# Set up Groq LLM (using Mixtral)
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=groq_api_key, model_name="meta-llama/llama-4-scout-17b-16e-instruct")

# Step 1: Define the state schema
from typing import TypedDict, Optional,Literal

class ResumeState(TypedDict):
    resume: str
    job_description: str
    analysis: Optional[str]
    gaps: Optional[str]
    enhanced_resume: Optional[str]

# Step 2: Define functions (LangChain Runnables)

def analyze_resume(state: ResumeState) -> ResumeState:
    prompt = f"""Analyze the following resume and list its key strengths and weaknesses:\n\n{state['resume']}"""
    result = llm.invoke(prompt)
    return {**state, "analysis": result.content}

def compare_resume_with_jd(state: ResumeState) -> ResumeState:
    prompt = f"""Given the resume:\n{state['resume']}\n\nAnd this job description:\n{state['job_description']}\n\nList the gaps, missing skills, and mismatch areas."""
    result = llm.invoke(prompt)
    return {**state, "gaps": result.content}

def enhance_resume(state: ResumeState) -> ResumeState:
    prompt = f"""Enhance the following resume:\n{state['resume']}\n\nBased on this job description:\n{state['job_description']}\n\nConsider these findings:\nAnalysis: {state['analysis']}\nGaps: {state['gaps']}"""
    result = llm.invoke(prompt)
    return {**state, "enhanced_resume": result.content}

# Step 3: Build LangGraph (Linear)
builder = StateGraph(ResumeState)

builder.add_node("AnalyzeResume", RunnableLambda(analyze_resume))
builder.add_node("CompareWithJD", RunnableLambda(compare_resume_with_jd))
builder.add_node("EnhanceResume", RunnableLambda(enhance_resume))

builder.set_entry_point("AnalyzeResume")
builder.add_edge("AnalyzeResume", "CompareWithJD")
builder.add_edge("CompareWithJD", "EnhanceResume")
builder.add_edge("EnhanceResume", END)

graph = builder.compile()

initial_state = {
    "resume": resume,
    "job_description": jd,
    "analysis": None,
    "gaps": None,
    "enhanced_resume": None,
}

final_state = graph.invoke(initial_state)

# Step 5: Output result
print("\n✅ Resume Analysis:\n", final_state["analysis"])
print("\n🔍 Job Match Gaps:\n", final_state["gaps"])
print("\n🚀 Enhanced Resume:\n", final_state["enhanced_resume"])

In [ ]:
from IPython.display import Markdown, display
display(Markdown(final_state["analysis"]))

## Second Iteration

In [ ]:
class ResumeState(TypedDict):
    resume: str
    job_description: str
    analysis: Optional[str]
    gaps: Optional[str]
    enhanced_resume: Optional[str]
    summary: Optional[str]
    path_decision: Optional[Literal["rewrite", "skip"]]


def evaluate_gaps(state: ResumeState) -> ResumeState:
    prompt = f"""Based on the following gap analysis between resume and job description, decide if the resume needs rewriting.
    If the resume is mostly aligned, return 'skip'. If there are many issues or gaps, return 'rewrite'.\n\nGaps:\n{state['gaps']}"""
    decision = llm.invoke(prompt).content.lower()
    if "rewrite" in decision:
        # Return the state with the decision added
        return {**state, "path_decision": "rewrite"}
    else:
        # Return the state with the decision added
        return {**state, "path_decision": "skip"}


def summarize_result(state: ResumeState) -> ResumeState:
    summary = f"""
Analysis:\n{state['analysis']}

 Gaps:\n{state['gaps']}

Final Resume:\n{state.get('enhanced_resume', 'Original resume retained.')}
"""
    return {**state, "summary": summary}


builder = StateGraph(ResumeState)

# Add nodes
builder.add_node("AnalyzeResume", RunnableLambda(analyze_resume))
builder.add_node("CompareWithJD", RunnableLambda(compare_resume_with_jd))
builder.add_node("EvaluateGaps", RunnableLambda(evaluate_gaps))
builder.add_node("EnhanceResume", RunnableLambda(enhance_resume))
builder.add_node("Summarize", RunnableLambda(summarize_result))

# Entry point
builder.set_entry_point("AnalyzeResume")

# Linear flow to start
builder.add_edge("AnalyzeResume", "CompareWithJD")
builder.add_edge("CompareWithJD", "EvaluateGaps")

# Conditional edge from EvaluateGaps
builder.add_conditional_edges(
    "EvaluateGaps",
    # Function that returns the path_decision from the state
    lambda state: state["path_decision"],
    {
        "rewrite": "EnhanceResume",
        "skip": "Summarize"
    }
)

# After enhancement, go to summary
builder.add_edge("EnhanceResume", "Summarize")

# End
builder.add_edge("Summarize", END)

graph = builder.compile()

final_state = graph.invoke(initial_state)

print("\n🎯 Summary:\n")
print(final_state["summary"])

In [ ]:
display(Markdown(final_state["summary"]))

## Third Iteration

In [ ]:
SERPER_API_KEY = "19ed5e2a267bec08c514815b99226d6c788603d7"

os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [ ]:
%%capture
!pip install langchain-community

In [ ]:
import os
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_groq import ChatGroq
from langchain_community.utilities import GoogleSerperAPIWrapper
# Load API Keys
load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

# Set up Groq LLM
llm = ChatGroq(api_key=groq_api_key, model_name="meta-llama/llama-4-scout-17b-16e-instruct")

# Define state schema
from typing import TypedDict, Optional, Literal

class ResumeState(TypedDict):
    resume: str
    job_description: str
    analysis: Optional[str]
    gaps: Optional[str]
    enhanced_resume: Optional[str]
    summary: Optional[str]
    path_decision: Optional[Literal["rewrite", "skip"]]
    job_suggestions: Optional[str]

# Define functions
def analyze_resume(state: ResumeState) -> ResumeState:
    prompt = f"Analyze the following resume and list its key strengths and weaknesses:\n\n{state['resume']}"
    result = llm.invoke(prompt)
    return {**state, "analysis": result.content}

def compare_resume_with_jd(state: ResumeState) -> ResumeState:
    prompt = f"Given the resume:\n{state['resume']}\n\nAnd this job description:\n{state['job_description']}\n\nList the gaps, missing skills, and mismatch areas."
    result = llm.invoke(prompt)
    return {**state, "gaps": result.content}

def evaluate_gaps(state: ResumeState) -> ResumeState:
    prompt = f"Based on the following gap analysis between resume and job description, decide if the resume needs rewriting. If mostly aligned, return 'skip'. If there are many issues, return 'rewrite'.\n\nGaps:\n{state['gaps']}"
    decision = llm.invoke(prompt).content.lower()
    return {**state, "path_decision": "rewrite" if "rewrite" in decision else "skip"}

def enhance_resume(state: ResumeState) -> ResumeState:
    prompt = f"Enhance this resume:\n{state['resume']}\n\nBased on this job description:\n{state['job_description']}\n\nFindings:\nAnalysis: {state['analysis']}\nGaps: {state['gaps']}"
    result = llm.invoke(prompt)
    return {**state, "enhanced_resume": result.content}

def summarize_result(state: ResumeState) -> ResumeState:
    summary = f"""
✅ Analysis:\n{state['analysis']}

🔍 Gaps:\n{state['gaps']}

🚀 Final Resume:\n{state.get('enhanced_resume', 'Original resume retained.')}

🧠 Job Suggestions:\n{state.get('job_suggestions', 'No suggestions found.')}
"""
    return {**state, "summary": summary}

# 🔍 Job Suggestion Agent using SERPER API
def fetch_job_suggestions(state: ResumeState) -> ResumeState:
    query = f"{state['job_description'].splitlines()[0]} jobs near me"
    search = GoogleSerperAPIWrapper(serper_api_key=serper_api_key)
    results = search.results(query)

    suggestions = []
    for result in results.get("organic", [])[:5]:
        title = result.get("title", "")
        link = result.get("link", "")
        suggestions.append(f"- {title}\n  🔗 {link}")

    jobs = "\n".join(suggestions) or "No relevant jobs found."
    return {**state, "job_suggestions": jobs}

# Build LangGraph
builder = StateGraph(ResumeState)

# Add nodes
builder.add_node("AnalyzeResume", RunnableLambda(analyze_resume))
builder.add_node("CompareWithJD", RunnableLambda(compare_resume_with_jd))
builder.add_node("EvaluateGaps", RunnableLambda(evaluate_gaps))
builder.add_node("EnhanceResume", RunnableLambda(enhance_resume))
builder.add_node("FetchJobs", RunnableLambda(fetch_job_suggestions))
builder.add_node("Summarize", RunnableLambda(summarize_result))

# Entry
builder.set_entry_point("AnalyzeResume")

# Flow
builder.add_edge("AnalyzeResume", "CompareWithJD")
builder.add_edge("CompareWithJD", "EvaluateGaps")

# Conditional enhancement or skip
builder.add_conditional_edges(
    "EvaluateGaps",
    lambda state: state["path_decision"],
    {
        "rewrite": "EnhanceResume",
        "skip": "FetchJobs"
    }
)

# After enhance, go to job fetch
builder.add_edge("EnhanceResume", "FetchJobs")

# Then to summary
builder.add_edge("FetchJobs", "Summarize")

# End
builder.add_edge("Summarize", END)

graph = builder.compile()

initial_state = {
    "resume": resume,
    "job_description": jd,
    "analysis": None,
    "gaps": None,
    "enhanced_resume": None,
    "path_decision": None,
    "summary": None,
    "job_suggestions": None,
}

final_state = graph.invoke(initial_state)

# Output
print("\n🎯 Final Summary:\n")
print(final_state["summary"])


In [ ]:
display(Markdown(final_state["summary"]))